<a href="https://colab.research.google.com/github/LabeebHameed/09/blob/main/Pulling_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-69kkmsx0
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-69kkmsx0
  Resolved https://github.com/tweepy/tweepy.git to commit db28c0e84826485755eb7fcef0c30f75395dff5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
pip install --upgrade tweepy

In [19]:
import tweepy
import json
import time

# ===================== AUTHENTICATION =====================
consumer_key = "qfBUM5GyhrZCKB1bQUTw5dpO6"
consumer_secret = "fUXk3NWmMk0fdPy6PdkabUz0pLjUPnk4hnR3cGnZuGdVVXA5To"
access_token = "1093844904281665536-TxqNmScf3109G1t2tUdxEqzSpYvpEL"
access_token_secret = "Pwgv5txQT2eWyv2VD6ab1yc6qMMEg6GLaEHXDZVSImv7O"

auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret,
    access_token, access_token_secret
)
api = tweepy.API(auth, wait_on_rate_limit=True)

# ===================== HELPER FUNCTIONS =====================

def get_all_ids(api_call, screen_name):
    ids = []
    for page in tweepy.Cursor(api_call, screen_name=screen_name, count=5000).pages():
        ids.extend(page)
        time.sleep(60)  # to avoid rate limits
    return ids

def save_ids(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f)

def load_ids(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

def compare_sets(prev, current):
    return list(set(prev) - set(current))  # unfollowers

# ===================== MAIN =====================

screen_name = "your_username_here"  # the authenticated user

print(f"\n📥 Fetching followers for @{screen_name}...")
followers = get_all_ids(api.followers_ids, screen_name)
print(f"✅ {len(followers)} followers fetched.")

print(f"\n📤 Fetching following for @{screen_name}...")
following = get_all_ids(api.friends_ids, screen_name)
print(f"✅ {len(following)} following fetched.")

# Save current data
save_ids("followers_current.json", followers)
save_ids("following_current.json", following)

# Load previous followers to detect unfollowers
prev_followers = load_ids("followers_previous.json")
unfollowers = compare_sets(prev_followers, followers)

# Save current as previous for next time
save_ids("followers_previous.json", followers)

# ===================== ANALYSIS =====================

followers_set = set(followers)
following_set = set(following)

mutuals = followers_set & following_set
dfmb = following_set - followers_set
fans = followers_set - following_set

# ===================== REPORT =====================
print("\n📊 RESULTS:")
print(f"🫂 Mutuals: {len(mutuals)}")
print(f"📤 DFMB (You follow, they don't): {len(dfmb)}")
print(f"📥 Fans (They follow, you don't): {len(fans)}")
print(f"❌ Unfollowers since last run: {len(unfollowers)}")

# Optional: save reports
save_ids("mutuals.json", list(mutuals))
save_ids("dfmb.json", list(dfmb))
save_ids("fans.json", list(fans))
save_ids("unfollowers.json", unfollowers)


Fetching data for @elonmusk (User ID: 44196397)...

Fetching followers...


Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

In [ ]:
tweets_df

In [ ]:
tweets[0]

In [ ]:
from google.colab import files
tweets_df.to_csv('tweets.csv')
files.download('tweets.csv')